In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import math
import os

import sys

import numpy as np
import pandas as pd
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import cv2
from keras.preprocessing.image import array_to_img
from PIL import Image as im 

In [ ]:
test_data=pd.read_csv("../input/img-dim-test/test.csv")
test_data.head()

In [ ]:
len(test_data)

In [ ]:

IMG_SIZE=512 # 256 , 1024 ...
rang=len(test_data)  # len(test_data) if we convert all data

for i in range (rang):
    voi_lut = True
    fix_monochrome = True
    image_id = test_data.loc[i, 'image_id'] # df insted of smalldata if we want all 
    
    dicom = pydicom.dcmread('../input/vinbigdata-chest-xray-abnormalities-detection/test/'+image_id+'.dicom')
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)

    #X_test =data_file.pixel_array
    new_array = cv2.resize(data, (IMG_SIZE,IMG_SIZE))# norm the value dont div on 255 
    new_array = np.reshape(new_array, (IMG_SIZE, IMG_SIZE,1))
    new_array = np.asarray(new_array)
    img = array_to_img(new_array)
    if (i+1)%300==0:
        print(i+1)

    img.save('/kaggle/working/' + image_id + '.jpg',dtype=np.uint8, quality=100) #or  .png